# Plot Crab Nebula SED (source-independent analysis)
Functions to create the plot of the LST-1 Crab Nebula SED from the source-independent analysis approach.

It includes:

  * LST-1 SED model (log-parabolic parametrization) from 50 GeV to 30 TeV. It includes the statistical uncertainty band. Solid line and band.
  * LST-1 & *Fermi*-LAT joint model (2 GeV to 2 TeV), also including the statistical uncertainty band. Dotted line.
  * SED flux points (from a DL3 sample obtained assuming 70% efficiency in *gammaness* and *theta* gamma-ray selection cuts). Filled circles markers.
  * Error band (hatched blue) encompassing statistical uncertainty bands from SEDs models based on datasets having different gamma-ray selection efficiencies. The efficiencies of the cuts were chosen in a grid of 40%-90% for *gammaness* and 70%-90% for angular cut (*theta*).
  * Flux points testing a +1% change in the background normalization, which only affects the low-energy spectral points (light open markers).

For reference:
  * *Fermi*-LAT Crab Nebula SED from Arakawa et al. (2020). Squared markers.
  * MAGIC Crab Nebula SED from Aleksić et al. (2015). Dashed line.

To plot only one or several of the above items, use the corresponding functions.

In [ ]:
import itertools
import pickle
from pathlib import Path

import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import seaborn as sns
from gammapy.datasets import FluxPointsDataset
from gammapy.estimators import FluxPoints
from gammapy.modeling.models import SkyModel, create_crab_spectral_model

In [ ]:
# Set plot style
style.use("seaborn-colorblind")

sns.set_color_codes("colorblind")
sns.set_palette("colorblind")

MPL_LINEWIDTH = 1.6

mpl_rc = {
    "figure.autolayout": False,
    "font.size": 20,
    "figure.dpi": 300,
    "lines.linewidth": 2.7,
    "axes.grid": False,
    "axes.linewidth": MPL_LINEWIDTH,
    "xtick.major.size": 8,
    "xtick.major.width": MPL_LINEWIDTH,
    "xtick.minor.size": 5,
    "xtick.minor.width": MPL_LINEWIDTH,
    "xtick.minor.visible": False,
    "ytick.major.size": 8,
    "ytick.major.width": MPL_LINEWIDTH,
    "ytick.minor.size": 5,
    "ytick.minor.width": MPL_LINEWIDTH,
    "ytick.minor.visible": True,
}
plt.style.use(mpl_rc)

In [ ]:
# Plotting kwargs and axis formatting

# For the MAGIC reference spectrum
crab_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.05, 30] * u.TeV,
    "yunits": u.Unit("erg cm-2 s-1"),
    "linestyle": "--",
}

# For the LST-1 only fit
sed_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.05, 30] * u.TeV,
}

# For the LST-1 flux points
lst1_flux_points_kwargs = {
    "sed_type": "e2dnde",
    "xerr": None,
    "color": "black",
    "markersize": 7,
    "zorder": 5,
}

# For the joint fit
sed_kwargs_joint_fit = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.002, 2] * u.TeV,
    "color": "g",
}

# For the hatch plot
sed_kwargs_hatch = {
    "sed_type": "e2dnde",
    "energy_bounds": [0.05, 30] * u.TeV,
    "facecolor": "w",
    "edgecolor": "b",
    "hatch": "//",
    "alpha": 1,
    "zorder": -1,
}

# For the test of +1% in background normalization
sed_kwargs_background_sys = {
    "sed_type": "e2dnde",
    "color": "lightgray",
    "marker": "o",
    "markersize": 7,
    "markeredgewidth": "2.4",
    "markerfacecolor": "white",
    "zorder": 4,
    "xerr": None,
    "label": r"$+1\%$ background systematics test (LST-1)",
}

kwargs_residuals = {"xerr": None, "markersize": 7}


def set_axes_sed(axis, xlabel=True):
    """Set axes limits and labels for the SED upper panel"""
    axis.set_xlim(1e-3, 4e1)  # 1 GeV to 40 TeV
    axis.set_ylim(1e-12, 3e-10)

    if xlabel:
        axis.set_xlabel(r"$E\,\,[{\rm TeV}]$")
    else:
        axis.set_xlabel(None)
    axis.set_ylabel(
        r"$E^2 \frac{{\rm d}\phi}{{\rm d} E}\, [{\rm erg}\,{\rm cm}^{-2}\,{\rm s}^{-1}]$"
    )


def set_axes_residuals(axis):
    """Set axes limits and labels for the residuals bottom panel"""
    axis.set_xlim(1e-3, 4e1)  # 1 GeV to 40 TeV
    axis.set_ylim(-1.1, 1.1)
    axis.set_yticks([-1, -0.5, 0, 0.5, 1], minor=True)

    axis.set_xlabel(r"$E\,\,[{\rm TeV}]$")
    axis.set_ylabel("Residuals\n(diff/model)")

In [ ]:
# Input files
BASE_PATH = Path("data")
src_indep_data = BASE_PATH / "src_indep"

fermi_lat_sed_file = BASE_PATH / "SED_Crab_FermiLAT_Arakawa2020.fits"
lst1_only_sed_model_file = src_indep_data / "SED_model_CrabNebula_only_LST1.dat"
lst1_fermi_joint_model_file = (
    src_indep_data / "SED_model_CrabNebula_joint_LST1_FermiLAT.dat"
)
lst1_flux_points_file = src_indep_data / "SED_CrabNebula_LST1_flux_points.fits"
lst1_flux_points_sys_bkg_file = src_indep_data / "test_background_sys_LST1.fits"

# Create directory for output plots
src_indep_plots = Path("src_indep_plots")
src_indep_plots.mkdir(exist_ok=True, parents=True)

In [ ]:
def get_fluxpoints_dataset(sed_file):
    """Returns two FluxPointsDatasets split according to the model they are based on (solo and joint fit).

    This way, it is easier to plot the residuals derived from both sets of flux points without overlapping.

    Notes
    -----
    SED flux points of LST-1 were calculated based on two different models, LST-1 only fit (50 GeV to 30 TeV)
    and LST-1+Fermi-LAT joint fit (2 GeV to 2 TeV). However, flux points were stored in the same FITS file
    for convenience. Points up to 2 TeV correspond to the joint model, while points above that energy
    are based on the former LST-1 only model.

    SED flux points from LST-1 are stored in the same FITS file.
    """
    lst1_flux_points = FluxPoints.read(sed_file, hdu="SED", format="gadf-sed")

    fp_table = lst1_flux_points.to_table()

    # Split flux points based on the energy range of the model
    fp_solo = fp_table[fp_table["e_max"] > 2 * u.TeV]
    fp_joint = fp_table[fp_table["e_max"] < 2 * u.TeV]

    # Build the FluxPoints object again based on both tables
    flux_points_lst1_solo = FluxPoints.from_table(fp_solo)
    flux_points_lst1_joint = FluxPoints.from_table(fp_joint)

    # Open the files with the models
    with open(lst1_only_sed_model_file, "rb") as flux_model:
        lst1_only_model_dict = pickle.load(flux_model)
    lst1_only_model = SkyModel.from_dict(lst1_only_model_dict)

    with open(lst1_fermi_joint_model_file, "rb") as flux_model:
        joint_model_dict = pickle.load(flux_model)
    joint_model = SkyModel.from_dict(joint_model_dict)

    # Create the datasets with flux points including the models
    flux_points_dataset_solo = FluxPointsDataset(
        models=lst1_only_model, data=flux_points_lst1_solo
    )
    flux_points_dataset_joint = FluxPointsDataset(
        models=joint_model, data=flux_points_lst1_joint
    )

    return flux_points_dataset_solo, flux_points_dataset_joint

In [ ]:
dataset_lst1_solo, dataset_lst1_joint_fermi = get_fluxpoints_dataset(
    lst1_flux_points_file
)

In [ ]:
def plot_fermi_ref(axis):
    """Plot spectrum from Fermi-LAT (Arakawa et al. 2020)"""
    fermi_flux_points = FluxPoints.read(fermi_lat_sed_file)

    fermi_flux_points.plot(
        label="$Fermi$-LAT (Arakawa et al. 2020)",
        color="m",
        marker="s",
        zorder=3,
        sed_type="e2dnde",
        ax=axis,
        markersize=6,
    )


def plot_magic_ref(axis):
    """Plot reference spectrum from MAGIC (Aleksić et al. 2015)"""
    create_crab_spectral_model("magic_lp").plot(
        **crab_kwargs,
        label="MAGIC (Aleksić et al. 2015)",
        color="r",
        ax=axis,
        zorder=2,
    )

In [ ]:
def plot_lst1_only_model(axis):
    with open(lst1_only_sed_model_file, "rb") as flux_model:
        lst1_only_model_dict = pickle.load(flux_model)
    lst1_only_model = SkyModel.from_dict(lst1_only_model_dict)

    lst1_only_model.spectral_model.plot(
        **sed_kwargs, ax=axis, label="Log-parabola fit (LST-1)"
    )
    lst1_only_model.spectral_model.plot_error(
        **sed_kwargs,
        ax=axis,
        facecolor="b",
        alpha=0.3,
        label="Stat. uncertainty (LST-1)",
        zorder=1,
    )


def plot_lst1_flux_points(axis):
    """Plot SED flux points from LST-1"""
    lst1_flux_points = FluxPoints.read(
        lst1_flux_points_file, hdu="SED", format="gadf-sed"
    )

    lst1_flux_points.plot(**lst1_flux_points_kwargs, ax=axis, label="LST-1 (this work)")


def plot_lst1_fermi_joint_model(axis):
    """Plot joint model from LST-1 and Fermi-LAT SED"""
    # Open the file
    with open(lst1_fermi_joint_model_file, "rb") as flux_model:
        joint_model_dict = pickle.load(flux_model)
    joint_model = SkyModel.from_dict(joint_model_dict)

    # Plot SED model
    joint_model.spectral_model.plot(
        **sed_kwargs_joint_fit,
        ax=axis,
        label="Log-parabola joint fit ($Fermi$-LAT and LST-1)",
        linestyle=":",
    )
    joint_model.spectral_model.plot_error(**sed_kwargs_joint_fit, ax=axis)


def plot_band_several_efficiencies(axis):
    """Plot SED band from several efficiencies in the gammaness and theta selection cuts"""
    gamma_efficiencies = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    theta_efficiencies = [0.7, 0.8, 0.9]

    for count, (gh_eff, th_eff) in enumerate(
        itertools.product(gamma_efficiencies, theta_efficiencies)
    ):
        file_model = (
            src_indep_data
            / f"effi_band/model_stacked_8bin_r2914_to_r7277_gh_eff_{gh_eff}_th_cont_{th_eff}.dat"
        )

        # Open files
        with open(file_model, "rb") as flux_model:
            model_dict = pickle.load(flux_model)

        model = SkyModel.from_dict(model_dict)

        if count == 0:
            model.spectral_model.plot_error(
                **sed_kwargs_hatch,
                label="Uncertainty from several\nefficiency cuts (LST-1)",
                ax=axis,
            )
        else:
            model.spectral_model.plot_error(**sed_kwargs_hatch, ax=axis)


def plot_test_background_normalization(axis):
    """Plot low energy SED points calculated assuming +1% background"""
    flux_points_table_sys = FluxPoints.read(lst1_flux_points_sys_bkg_file)
    flux_points_table_sys.plot(ax=axis, **sed_kwargs_background_sys)


def plot_residuals(ax, dataset, **kwargs):
    """Plot residuals

    Based on the `plot_residuals` method of `FluxPointsDataset` allowing to set
    xerr=None which is not possible in the current Gammapy implementation.
    See https://docs.gammapy.org/dev/_modules/gammapy/datasets/flux_points.html#FluxPointsDataset

    Parameters
    ----------
    ax : `~matplotlib.axes.Axes`
        Axes to plot on
    dataset : `FluxPointsDataset`
        Datasets with flux points
    **kwargs : dict
        Keyword arguments passed to `~matplotlib.axes.Axes.errorbar`
    """
    method = "diff/model"  # We just consider this model here
    fp = dataset.data
    residuals = dataset.residuals(method)

    xerr = dataset.data.energy_axis.as_plot_xerr
    yerr = fp._plot_get_flux_err(sed_type="dnde")

    # Considering `diff/mode`
    model = dataset.flux_pred()
    yerr = (yerr[0].quantity[:, 0, 0] / model), (yerr[1].quantity[:, 0, 0] / model)

    kwargs.setdefault("xerr", xerr)
    kwargs.setdefault("yerr", yerr)
    kwargs.setdefault("color", kwargs.pop("c", "black"))
    kwargs.setdefault("fmt", "o")

    ax.errorbar(fp.energy_ref, residuals, **kwargs)

    # format axes
    ax.set_xscale("log")
    ymin = np.nanmin(residuals - yerr[0])
    ymax = np.nanmax(residuals + yerr[1])
    ymax = max(abs(ymin), ymax)
    ax.set_ylim(-1.05 * ymax, 1.05 * ymax)

    ax.axhline(0, color=kwargs["color"], lw=0.5)

In [ ]:
# Make the plot
fig, ax = plt.subplots(2, 1, gridspec_kw={"height_ratios": [5, 1]}, figsize=(12, 12))

# Plot LST-1 results
plot_lst1_flux_points(ax[0])
plot_lst1_only_model(ax[0])
plot_lst1_fermi_joint_model(ax[0])

# Plot MAGIC and Fermi-LAT SEDs
plot_fermi_ref(ax[0])
plot_magic_ref(ax[0])

# Test several efficiencies
plot_band_several_efficiencies(ax[0])
# Test a change of +1% in background normalization
plot_test_background_normalization(ax[0])

# Plot residuals
plot_residuals(ax[1], dataset_lst1_solo, **kwargs_residuals)
plot_residuals(ax[1], dataset_lst1_joint_fermi, **kwargs_residuals)

# Axes format, limits and labels
set_axes_sed(ax[0], xlabel=False)
set_axes_residuals(ax[1])

# Reorder the labels in the legend
handles, labels = ax[0].get_legend_handles_labels()
order = [5, 0, 1, 7, 4, 2, 3, 6]
ax[0].legend(
    [handles[idx] for idx in order], [labels[idx] for idx in order], loc="lower left"
)

plt.savefig(
    src_indep_plots / "lst1_crab_sed_src_indep.png", bbox_inches="tight"
)
plt.savefig(
    src_indep_plots / "lst1_crab_sed_src_indep.pdf", bbox_inches="tight"
)